In [1]:
%load_ext autoreload

%autoreload 2

In [12]:
%matplotlib inline
import os
import sys
sys.path.append('..')

from scripts.data_utils.cleaner import *
from scripts.modeling.process import process_data
from scripts.data_utils.preprocess import preprocess_data
from scripts.data_utils.loaders import load_data, save_data
from scripts.modeling.model import prepare_for_modelling, CreditScoringModel

# import matplotlib
# matplotlib.use('TkAgg')

In [3]:
RESOURCEPATH = os.path.join('..', 'resources')
DATAPATH = os.path.join(RESOURCEPATH, 'data')
raw_data_path = os.path.join(DATAPATH, 'raw')

prepreocessed_output_dir = os.path.join(DATAPATH, 'preprocessed')
processed_output_dir = os.path.join(DATAPATH, 'processed')
plot_output_dir = os.path.join('..', 'screenshots', 'plots')

os.makedirs(prepreocessed_output_dir, exist_ok=True)
os.makedirs(processed_output_dir, exist_ok=True)
os.makedirs(plot_output_dir, exist_ok=True)

In [4]:
filename = 'data.csv'
file_path = os.path.join(raw_data_path, filename)
data_csv = load_data(file_path)
data_csv.shape

2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:25 - INFO - Loading data from ..\resources\data\raw\data.csv
2025-01-29 07:17:26 - INFO - Successfully loaded data from ..\resources\data\raw\data.csv
2025-01-29 07:17:26 - INFO - Successfully loaded data from ..\resources\data\raw\data.csv
2025-01-29 07:17:26 - INFO - Successfully loaded data from ..\resources\data\raw\data.csv
2025-01-29 07:17:26 - INFO -

(95662, 16)

In [5]:
filename2 = 'data.xlsx'
file_path2 = os.path.join(raw_data_path, filename2)
data_xlsx = load_data(file_path2, sheet_name='data')
data_xlsx.shape

2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:17:27 - INFO - Loading data from ..\resources\data\raw\data.xlsx
2025-01-29 07:18:05 - INFO - Successfully loaded data from ..\resources\data\raw\data.xlsx
2025-01-29 07:18:05 - INFO - Successfully loaded data from ..\resources\data\raw\data.xlsx
2025-01-29 07:18:05 - INFO - Successfully loaded data from ..\resources\data\raw\data.xlsx
2025-01-29 07:18

(95662, 18)

In [6]:
data = data_xlsx.copy()

[autoreload of scripts.modeling.model failed: Traceback (most recent call last):
  File "c:\Users\Darkles\10academy\env\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Darkles\10academy\env\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "C:\Users\Darkles\AppData\Local\Programs\Python\Python312\Lib\importlib\__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 994, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "c:\Users\Darkles\10academy\10Academy-Kifiya-Week-6\notebooks\..\scripts\modeling\model.py", line 158, in <module>
    X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=[target_column]), data_final

# Preprocessed

In [7]:
irrelevant_columns = ['Unnamed: 16', 'Unnamed: 17']
numerical_columns = ['Amount', 'Value', 'PricingStrategy']
categorical_columns = ["CurrencyCode", "CountryCode", "ProviderId", "ProductId", "ProductCategory", "ChannelId"]
date_column = "TransactionStartTime"

missing_value_strategies = {
        "CountryCode": "most_frequent",
        "AccountId": "most_frequent",
        "ProviderId": "most_frequent",
        "PricingStrategy": "median",
        "Value": "mean",
}
dtype_conversions = {
        "CountryCode": "int64",
        "CountryCode": "str",
}

data = data_xlsx.copy()

data_preprocessed = preprocess_data(data, irrelevant_columns, categorical_columns, numerical_columns, 
                                    missing_value_strategies, date_column, dtype_conversions, processed_output_dir)
data_preprocessed

2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Cleaning...
2025-01-29 07:18:06 - INFO - Dropped columns: {'Unnamed: 16', 'Unnamed: 17'}. Remaining columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult']
2025-01-29 07:18:06 - INFO - Dropped columns: {'Unnamed: 16', 'Unnamed: 17'}. Remaining columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId', 'ProductCategory', 'Cha

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256.0,PROVIDERID_6,PRODUCTID_10,AIRTIME,CHANNELID_3,1000.0,1000.00000,2018-11-15 05:18:49+03:00,2.0,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256.0,PROVIDERID_4,PRODUCTID_6,FINANCIAL_SERVICES,CHANNELID_2,3679.0,20.00000,2018-11-15 05:19:08+03:00,2.0,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256.0,PROVIDERID_4,PRODUCTID_1,AIRTIME,CHANNELID_3,500.0,500.00000,2018-11-15 05:44:21+03:00,2.0,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256.0,PROVIDERID_1,PRODUCTID_21,UTILITY_BILL,CHANNELID_3,20000.0,9900.64041,2018-11-15 06:32:55+03:00,2.0,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256.0,PROVIDERID_4,PRODUCTID_6,FINANCIAL_SERVICES,CHANNELID_2,-644.0,644.00000,2018-11-15 06:34:21+03:00,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95656,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256.0,PROVIDERID_4,PRODUCTID_6,FINANCIAL_SERVICES,CHANNELID_2,-1000.0,1000.00000,2019-02-13 12:54:09+03:00,2.0,0
95657,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256.0,PROVIDERID_6,PRODUCTID_10,AIRTIME,CHANNELID_3,1000.0,1000.00000,2019-02-13 12:54:25+03:00,2.0,0
95658,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256.0,PROVIDERID_4,PRODUCTID_6,FINANCIAL_SERVICES,CHANNELID_2,-20.0,20.00000,2019-02-13 12:54:35+03:00,2.0,0
95659,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256.0,PROVIDERID_6,PRODUCTID_19,TV,CHANNELID_3,3000.0,3000.00000,2019-02-13 13:01:10+03:00,2.0,0


# Feature Engineering

In [9]:
# Define the target feature and feature lists
drop_columns = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId']
numerical_columns = ['Amount', 'Value', 'PricingStrategy']
temporal_features = ['transaction_hour', 'transaction_day', 'transaction_month', 'transaction_year']
categorical_columns = ["CurrencyCode", "CountryCode", "ProviderId", "ProductId", "ProductCategory", "ChannelId"]
aggregated_features = ['total_transaction_amount', 'avg_transaction_amount', 'transaction_count', 'std_transaction_amount']
numerical_features = numerical_columns + aggregated_features
columns = categorical_columns + numerical_features

rfms_features = ['Recency', 'Frequency', 'Monetary', 'Severity']
score_column = rfms_features[2]
cluster_column = rfms_features[0]
date_column = 'TransactionStartTime'
target_column = 'FraudResult'
customer_column = "CustomerId"
label_column = "RFMS_Label"
recency_column = date_column
frequency_column = date_column
monetary_column = "Amount"
severity_column = "Value"

data = data_preprocessed.copy()
scaler= None
data_processed = process_data(data, numerical_features, date_column, customer_column, recency_column, frequency_column, monetary_column, 
             severity_column, target_column, label_column, columns, rfms_features, scaler, processed_output_dir)

2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Cleaning...
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO - Creating aggregate features.
2025-01-29 07:18:15 - INFO 

Index(['CustomerId', 'Recency', 'Frequency', 'Monetary', 'Severity'], dtype='object') ['Frequency', 'Monetary', 'Recency', 'Severity']


c:\Users\Darkles\10academy\10Academy-Kifiya-Week-6\notebooks\..\scripts\utils\visualization.py:123: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - Applying WoE binning.
2025-01-29 07:18:17 - INFO - WoE binning and IV calculation completed for CurrencyCode.
2025-01-29 07:18:17 - INFO - WoE binning and IV calculation completed for CurrencyCode.
2025-01-29 07:18:17 - INFO - WoE binning and IV calculation completed for Cu

Model

In [10]:
drop_columns = ['TransactionStartTime', 'TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId']
target_columns = 'FraudResult'
cluster_label = 'RFMS_Label'
data_final = prepare_for_modelling(data_processed, cluster_label, drop_columns)
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95661 entries, 0 to 95660
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CurrencyCode              95661 non-null  float64
 1   CountryCode               95661 non-null  float64
 2   ProviderId                95661 non-null  float64
 3   ProductId                 95661 non-null  float64
 4   ProductCategory           95661 non-null  float64
 5   ChannelId                 95661 non-null  float64
 6   Amount                    95661 non-null  float64
 7   Value                     95661 non-null  float64
 8   PricingStrategy           95661 non-null  float64
 9   FraudResult               95661 non-null  int64  
 10  total_transaction_amount  95661 non-null  float64
 11  avg_transaction_amount    95661 non-null  float64
 12  transaction_count         95661 non-null  float64
 13  std_transaction_amount    95661 non-null  float64
 14  transa

In [ ]:
# # Split data to train and test
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=[target_column]), data_final[target_column], test_size=0.2)
# data_training = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
# data_training

In [14]:
# Initialize and run the model pipeline
credit_model = CreditScoringModel(data_final, target_column=target_column)
best_credit_scoring_model, test_data = credit_model.run_pipeline()
best_credit_scoring_model

2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:11 - INFO - Preprocessing data...
2025-01-29 07:52:12 - INFO - Train shape: (68875, 23), Validation shape: (17219, 23)
2025-01-29 07:52:12 - INFO - Train shape: (68875, 23), Validation shape: (17219, 23)
2025-01-29 07:52:12 - INFO - Train shape: (68875, 23), Validation shape: (17219, 23)
2025-01-29 07:52:12 - INFO - Tr

RandomForestClassifier(max_depth=10)

In [15]:
# Evaluate
X_test, y_test = test_data
metrics = credit_model.evaluate_model(best_credit_scoring_model, X_test, y_test)
metrics

{'accuracy': 0.9984321103794292,
 'precision': 0.7777777777777778,
 'recall': 0.35,
 'f1_score': 0.4827586206896552,
 'roc_auc': 0.996925735833246}